In [ ]:

# Make Predictions with k-nearest neighbors on the Iris Flowers Dataset
from csv import reader
from math import sqrt
 
# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset
 
'''# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
		print('[%s] => %d' % (value, i))
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup
 
# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	for i in range(len(dataset[0])):
		col_values = [row[i] for row in dataset]
		value_min = min(col_values)
		value_max = max(col_values)
		minmax.append([value_min, value_max])
	return minmax
 
# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)):
			row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
 '''
# Calculate the Euclidean distance between two vectors
def euclidean_distance(row1, row2):
	distance = 0.0
	for i in range(len(row1)-1):
		distance += (row1[i] - row2[i])**2
	return sqrt(distance)
 
# Locate the most similar neighbors
def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors
 
# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction
 
# Make a prediction with KNN on Iris Dataset
'''filename = 'iris.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)'''
# define model parameter
num_neighbors = 5
# define a new record
row = [5.7,2.9,4.2,1.3]
# predict the label
label = predict_classification(dataset, row, num_neighbors)
print('Data=%s, Predicted: %s' % (row, label))

In [1]:
!pip install ipython-autotime
%load_ext autotime

time: 1.88 ms (started: 2022-03-09 06:11:55 +00:00)


In [2]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-91a58f43-80b2-2428-dc89-cf0549b86ce2)
time: 120 ms (started: 2022-03-09 06:12:00 +00:00)


In [3]:
!pip install nufhe[pycuda]
!pip install pycuda

     |████████████████████████████████| 86 kB 2.5 MB/s 
     |████████████████████████████████| 191 kB 18.4 MB/s 
     |████████████████████████████████| 1.7 MB 48.1 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 75 kB 4.3 MB/s 
     |████████████████████████████████| 67 kB 5.8 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626634 sha256=13661b27c66dc22325614792c0f2213dc44fb4d05e9f7cfcb33d49df2229ddf2
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2022.1.1-py2.py3-none-any.whl size=63859 sha256=a20c76c01a347e8403084e9f2cdd02b8416934e9f7b675601a7730493415c1c2
  Stored in directory: /root/.cache/pip/wheels/ee/a2/73/6f43d4f99ec69f7731617d44b68bf85aa66fc2da7b33cbe16b
  Created wheel for reikna: filename=reikna-0.7.6-py3-

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 19.1 s (started: 2022-03-09 06:16:32 +00:00)


KNN Brast cancer

1.   List item
2.   List item




In [26]:
import time
import random
#import pycuda.autoinit
import nufhe
import numpy as np
import numpy as np
from datetime import datetime
def subtractBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)      #axorb
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry) #axorbxorc
    # And(t1[0], t1[0], t2[0])
    acomp=vm.gate_not(a)   #a'
    abcomp=vm.gate_not(t1)  #axorb'
    t2 = vm.gate_and(acomp, b)
    t3 = vm.gate_and(abcomp, carry)
    #t4=vm.gate_and(t2,t3)
    #t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t2, t3)

    return r
def subtractNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(0,nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    t1=vm.gate_not(ctA[0])
    carry = vm.gate_and(t1, ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
        bitResult = subtractBits(bitResult, ctA[i], ctB[i], carry)
        # Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])

        # Copy(carry[0], bitResult[1])
        carry = nufhe.LweSampleArray.copy(bitResult[1])

    return ctRes

def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction
#def bubblesort(distances)

def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row,nbits)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors




def euclidean_distance(row1, row2,nBits):
    ones= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    distance=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    c_zero= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    #distance = 0.0
    '''for i in range(nBits):
        temp=vm.gate_copy(row2[i])
        ones[i]=vm.gate_not(temp)
    neg2=make_neg(ones,nBits)'''
    
    
    result=subtractNumbers(row1,row2,nBits)
    signbit=vm.gate_copy(result[len(result)-1])
    for i in range(nBits):
        temp=vm.gate_copy(result[i])
        ones[i]=vm.gate_not(temp)
    neg2=make_neg(ones,nBits)      
    for i in range(0,nBits):
           distance[i]=vm.gate_mux(signbit,neg2[i],result[i])
    return distance
def subtractNumbers_2s(ctA, ctB, nBits):
    ctA.reverse()
    ctB.reverse()
    temp1=ctA[:]
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print(pa)
    print("CAta in 2s",twos_comp_val(int(pa,2),len(pa)))
    temp1=ctB[:]
    
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print(pa)
    print("catb in 2s",twos_comp_val(int(pa,2),len(pa)))



    ones= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    c_zero= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(0,nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    t1=vm.gate_not(ctA[0])
    carry = vm.gate_and(t1, ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
        bitResult = subtractBits(bitResult, ctA[i], ctB[i], carry)
        # Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])

        # Copy(carry[0], bitResult[1])
        carry = nufhe.LweSampleArray.copy(bitResult[1])
    temp1=ctRes[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print(pa)
    print("subtraction for decession",twos_comp_val(int(pa,2),len(pa)))
    return ctRes[size-1]
    
def make_neg(n,nbits):
    list1=[int(i) for i in range(0,len(n)) ]
    listp=[]
    #one=vm.empty_ciphertext((1,))
    #zero=[vm.empty_ciphertext((1,))]
    #one= [ctx.encrypt(secret_key, [True])]
    #zero= [ctx.encrypt(secret_key, [False])]
    #t2= [ctx.encrypt(secret_key, [False])]
    #one=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    #zero=  [ctx.encrypt(secret_key, [False) for _ in range(nbits)]
    #print("type of one is",type(one))
    #print("typeof zero is",type(zero))
    #print("tupe of one is",one)
    #[vm.empty_ciphertext((1,))]
    '''for i in range(0,len(n)):
        #temp=vm.gate_copy(n[i])
        temp=vm.gate_mux(n[i],zero[0],one[0])
        n[i]=temp[:]'''
            
    
    #print(n)
    one= ctx.encrypt(secret_key, [True])
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    onep[0]=one 
    testone= [[vm.empty_ciphertext((1,))] for i in range(nbits)]
    testone=onep[:]
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n before",twos_comp_val(int(pa,2),len(pa)))'''
    n=addNumbers(n,testone,nbits)
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n after",twos_comp_val(int(pa,2),len(pa)))'''
    return n


def fixSizeBoolList(decimal,size):
    x = [int(x) for x in bin(decimal)[2:]]
    x = list(map(bool, x))
    x = [False]*(size - len(x)) + x
    pow2 = []
    for i in range(size):
        pow2.append([x[i]])
    pow2.reverse()
    return pow2
def boolListToInt(bitlists):
    out = 0
    for bit in bitlists:
        out = (out << 1) | bit
    return out

def addBits_prop(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    r[1] = vm.gate_and(t1, carry)
def addBits_approx(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_and(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_or(t1, carry)
    # And(t1[0], t1[0], t2[0])
    #t2 = vm.gate_and(a, carry)
    #t3 = vm.gate_and(b, carry)
    #t4=vm.gate_and(a,b)
    #t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(a, b)

    return r
def addBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, b)
    t3 = vm.gate_and(t1, carry)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t2, t3)

    return r
def addNumbers_prop(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    carry = vm.gate_and(ctA[0], ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
        if i>0:
            bitResult = addBits_approx(bitResult, ctA[i], ctB[i], carry)
            # Copy(ctRes[i], bitResult[0]);
            ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
            # Copy(carry[0], bitResult[1])
            carry = nufhe.LweSampleArray.copy(bitResult[1])
        else:
            #bitResult = twotoone(bitResult, ctA[i], ctB[i])
            bitResult = threetotwo(bitResult, ctA[i], ctB[i],carry)
            #Copy(ctRes[i], bitResult[0]);
            ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
            carry = nufhe.LweSampleArray.copy(bitResult[1])    
def addNumbers_approx(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    carry = vm.gate_and(ctA[0], ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
        if i>0:
            bitResult = addBits_approx(bitResult, ctA[i], ctB[i], carry)
            # Copy(ctRes[i], bitResult[0]);
            ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
            # Copy(carry[0], bitResult[1])
            carry = nufhe.LweSampleArray.copy(bitResult[1])
        else:
            #bitResult = twotoone(bitResult, ctA[i], ctB[i])
            bitResult = threetotwo(bitResult, ctA[i], ctB[i],carry)
            #Copy(ctRes[i], bitResult[0]);
            ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
            carry = nufhe.LweSampleArray.copy(bitResult[1])
        
    return ctRes    
def addNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    carry = vm.gate_and(ctA[0], ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
        if i>0:
            bitResult = addBits(bitResult, ctA[i], ctB[i], carry)
            # Copy(ctRes[i], bitResult[0]);
            ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
            # Copy(carry[0], bitResult[1])
            carry = nufhe.LweSampleArray.copy(bitResult[1])
        else:
            #bitResult = twotoone(bitResult, ctA[i], ctB[i])
            bitResult = threetotwo(bitResult, ctA[i], ctB[i],carry)
            #Copy(ctRes[i], bitResult[0]);
            ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
            carry = nufhe.LweSampleArray.copy(bitResult[1])
        
    return ctRes        
def Convert_list(string):
    list1=[]
    list1[:0]=string
    #print(list1)
    list1=[int(i)for i in list1 ]
    listb=[]
    for i in list1:
        if i==0:
            listb.append([False])
        else:
            listb.append([True])    
    
    #print(listb)
    return listb

def twos_complement(n,nbits):
    a=f"{n & ((1 << nbits) - 1):0{nbits}b}" 
    #print(type(a))
    a=Convert_list(a)
    a.reverse()
    return a
def listToString(s): 
    # initialize an empty string
    list1=[int(i)for i in s ]
    listp=[]
    for i in list1:
        if i==False:
            listp.append('0')
        else:
            listp.append('1')    
    
    #print(listp)
    str1 = "" 
    # traverse in the string  
    s=['delim'.join([str(elem) for elem in sublist]) for sublist in listp]
    #print(s)
    for ele in s: 
        str1 += ele  
    # return string 
    #print(str1)
    return str1
def twos_comp_val(val,bits):
    """compute the 2's complement of int value val"""
    #val=listToString(val)
    

    if (val & (1 << bits - 1)) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val

def compare_bit(  a, b,  lsb_carry,  tmp): 
    result= ctx.encrypt(secret_key, [False])
    tmp=vm.gate_xnor(a, b)
    result=vm.gate_mux(tmp,lsb_carry, a)
    return result


def minimum(  a,  b,  nb_bits): 
    tmps1= ctx.encrypt(secret_key, [False])
    tmps2= ctx.encrypt(secret_key, [True])
    #initialize the carry to 0
    #run the elementary comparator gate n times
    for i in range(nb_bits):
        tmps1= compare_bit(a[i],b[i],tmps1,tmps2)
    #tmps[0] is the result of the comparaison: 0 if a is larger, 1 if b is larger
    #select the max and copy it to the result
    return tmps1

    
        
def predict(ctA,secret_key, output_bits):
    zero = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    onen=  [ctx.encrypt(secret_key, [True]) for _ in range(output_bits)]        
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    one= ctx.encrypt(secret_key, [True])
    temp= ctx.encrypt(secret_key, [True])
    temp=ctA[output_bits-1]
    comp_res= ctx.encrypt(secret_key, [True])
    onep[0]=one
    ctRes = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    # Copy(ctRes[i], bitResult[0]);
    #comp_res= minimum(ctA,zero,output_bits)
    #temp=
    #comp_res=
    
    
    
    for i in range(output_bits):
        ctRes[i] = vm.gate_mux(temp,onen[i],onep[i])  
    # Copy(carry[0], bitResult[1])
    return ctRes

















if __name__ == '__main__':    
    ctx = nufhe.Context()
    secret_key, cloud_key = ctx.make_key_pair()
    vm = ctx.make_virtual_machine(cloud_key)
    size=16
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    onen=  [ctx.encrypt(secret_key, [True]) for _ in range(size)]
    temp_dist=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    temp_label=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    dist_min=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    dist_max=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    one= ctx.encrypt(secret_key, [True])
    zero=ctx.encrypt(secret_key, [False])
    onep[0]=vm.gate_copy(one)
    with open('/content/drive/MyDrive/hearttrain.txt') as f:
        lines = []
        for line in f:
            lines.append([int(v) for v in line.split()])
    with open('/content/drive/MyDrive/hearttest.txt') as f:
        lines1 = []
        for line in f:
            lines1.append([int(v) for v in line.split()])
            print(line)
    bits = [[False] for i in range(size - 2)]
    zeros = [[False] for i in range(size)]
    train_size= int(input("Please enter train a string:\n"))
    test_size=int(input("Please enter test data a string:\n"))
    neigh=int(input("Please enter no of neighbours:\n"))
    

    '''b_x0=[]
    b_x1=[]
    b_x2=[]
    b_x3=[]
    for i in range(test_size):
        temp=int(lines[i][0])
        #print(type(temp))
        b_x0.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][1])
        b_x1.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][2])
        b_x2.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][3])
        b_x3.append(fixSizeBoolList(temp,size))'''
    #print(b_x0,b_x1,b_x2,b_x3)
    #-3.5133618, -4.2616279 , 10.59763759,  6.17913037, -0.1758181
    #w_p=[-15,-18 ,-78 ,138 , 62] # scratch x/1+|x|
    w_p=[-1,-3,4,2,0] # sk learn x/1+|x|
    #w_p=[4,-8 ,22 ,9 , -1] sigmoid
    #w_p=[-3, -4 , 10,  6, -1] svm
    
    #print(type(x[0]))
    #print(x)
    #b_y = fixSizeBoolList(deci_y,size)
    #print(type(y[0]))
    #print(y)
    #x.reverse()
    #print(x)
    #y.reverse()
    #print(y)
    featuresize=5
    '''w=[]
    w_b=[]
    for i in range(featuresize-1):
        w.append([[vm.empty_ciphertext((1,))] for i in range(size)])
        w_b.append(twos_complement(w_p[i],size))
        for j in range(size):
            w[i][j] = ctx.encrypt(secret_key, w_b[i][j])
    bias=[[vm.empty_ciphertext((1,))] for i in range(size)]
    bias_b=twos_complement(w_p[4],size)'''
    '''eten=[[vm.empty_ciphertext((1,))] for i in range(size)]
    eten_b=twos_complement(10,size)
    for i in range(size):
        bias[i]=ctx.encrypt(secret_key, bias_b[i])
        eten[i]=ctx.encrypt(secret_key, eten_b[i])
    '''
    #train_data = pd.read_csv('/home/pradeep/Desktop/train.txt',sep=" ",header = None)
    #test_data=pd.read_csv('/home/pradeep/Desktop/test.txt',sep=" ",header = None)
    #dist=[]
    #for i in range(train_size):
    #    dist.append([[ctx.encrypt(secret_key, [False]) for i in range(16)] for j in range(2)]) 
    dist=[[ctx.encrypt(secret_key, [False]) for i in range(size)] for j in range(train_size+1)]
    label=[[ctx.encrypt(secret_key, [False]) for i in range(size)] for j in range(train_size+1)]
    plain_predict=[]
    temp=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
    temp_s1=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
    temp_s2=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
    temp_s3=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
    temp_s4=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
    start_time = datetime.now()
    #dist=[[vm.empty_ciphertext((1,))] for i in range(train_size) for j in range(size)]
    ciphertextl=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext1a=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext2a=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext3a=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext4a=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext5a=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext6a=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext7a=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext8a=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext9a=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext10a=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext11a=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext12a=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext13a=[[vm.empty_ciphertext((1,))] for i in range(size)]

    ciphertext1=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext2=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext3=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext4=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext5=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext6=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext7=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext8=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext9=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext10=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext11=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext12=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertext13=[[vm.empty_ciphertext((1,))] for i in range(size)]
    ciphertextl=[[vm.empty_ciphertext((1,))] for i in range(size)]

    pos=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
    neg=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
    pos_l=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
    neg_l=[ctx.encrypt(secret_key, [True]) for _ in range(size)]
    onebit=ctx.encrypt(secret_key, [True])
    pos_l[0]=onebit
    predict1=[ctx.encrypt(secret_key, [False]) for i in range(size)] 
    posc=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
    negc=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
    one=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
    one[0]=onebit
    temp1 = [[vm.empty_ciphertext((1,))] for i in range(size)]
    temp2 = [[vm.empty_ciphertext((1,))] for i in range(size)]
    
    for j in range(0,test_size):
        
        print("j values is",j)
        print("################################")
        temp1=int(lines1[j][0])
        b_y0=twos_complement(temp1,size)
        temp1=int(lines1[j][1])
        b_y1=twos_complement(temp1,size)
        temp1=int(lines1[j][2])
        b_y2=twos_complement(temp1,size)
        temp1=int(lines1[j][3])
        b_y3=twos_complement(temp1,size)
        temp1=int(lines1[j][4])
        b_y4=twos_complement(temp1,size)
        temp1=int(lines1[j][5])
        b_y5=twos_complement(temp1,size)
        temp1=int(lines1[j][6])
        b_y6=twos_complement(temp1,size)
        temp1=int(lines1[j][7])
        b_y7=twos_complement(temp1,size)
        temp1=int(lines1[j][8])
        b_y8=twos_complement(temp1,size)
        temp1=int(lines1[j][9])
        b_y9=twos_complement(temp1,size)
        temp1=int(lines1[j][10])
        b_y10=twos_complement(temp1,size)
        temp1=int(lines1[j][11])
        b_y11=twos_complement(temp1,size)
        temp1=int(lines1[j][12])
        b_y12=twos_complement(temp1,size)
        for _ in range(size):
                ciphertext1a[_] = ctx.encrypt(secret_key, b_y0[_])
                ciphertext2a[_] = ctx.encrypt(secret_key, b_y1[_])
                ciphertext3a[_] = ctx.encrypt(secret_key, b_y2[_])
                ciphertext4a[_] = ctx.encrypt(secret_key, b_y3[_])
                ciphertext5a[_] = ctx.encrypt(secret_key, b_y4[_])
                ciphertext6a[_] = ctx.encrypt(secret_key, b_y5[_])
                ciphertext7a[_] = ctx.encrypt(secret_key, b_y6[_])
                ciphertext8a[_] = ctx.encrypt(secret_key, b_y7[_])
                ciphertext9a[_] = ctx.encrypt(secret_key, b_y8[_])
                ciphertext10a[_] = ctx.encrypt(secret_key, b_y9[_])
                ciphertext11a[_] = ctx.encrypt(secret_key, b_y10[_])
                ciphertext12a[_] = ctx.encrypt(secret_key, b_y11[_])
                ciphertext13a[_] = ctx.encrypt(secret_key, b_y12[_])
        print("test data")
        print(lines1[j][0],lines1[j][1],lines1[j][2],lines1[j][3],lines1[j][4],lines1[j][5],lines1[j][6],lines1[j][7],lines1[j][8],lines1[j][9],lines1[j][10],lines1[j][11],lines1[j][12])
        for i in range(train_size):
            print("i value is ",i)
            print("training data")
            print(lines[i][0],lines[i][1],lines[i][2],lines[i][3],lines[i][4],lines[i][5],lines[i][6],lines[i][7],lines[i][8],lines[i][9],lines[i][10],lines[i][11],lines[j][12],lines[j][13])
            
            temp=int(lines[i][0])
            b_x0=twos_complement(temp,size)
            temp=int(lines[i][1])
            b_x1=twos_complement(temp,size)
            temp=int(lines[i][2])
            b_x2=twos_complement(temp,size)
            temp=int(lines[i][3])
            b_x3=twos_complement(temp,size)
            temp=int(lines[i][4])
            b_x4=twos_complement(temp,size)
            temp=int(lines[i][5])
            b_x5=twos_complement(temp,size)
            temp=int(lines[i][6])
            b_x6=twos_complement(temp,size)
            temp=int(lines[i][7])
            b_x7=twos_complement(temp,size)
            temp=int(lines[i][8])
            b_x8=twos_complement(temp,size)
            temp=int(lines[i][9])
            b_x9=twos_complement(temp,size)
            temp=int(lines[i][10])
            b_x10=twos_complement(temp,size)
            temp=int(lines[i][11])
            b_x11=twos_complement(temp,size)
            temp=int(lines[i][12])
            b_x12=twos_complement(temp,size)
            
            temp=int(lines[i][13])
            b_l=twos_complement(temp,size)
            
            for j in range(size):
                ciphertext1[j] = ctx.encrypt(secret_key, b_x0[j])
                ciphertext2[j] = ctx.encrypt(secret_key, b_x1[j])
                ciphertext3[j] = ctx.encrypt(secret_key, b_x2[j])
                ciphertext4[j] = ctx.encrypt(secret_key, b_x3[j])
                ciphertext5[j] = ctx.encrypt(secret_key, b_x4[j])
                ciphertext6[j] = ctx.encrypt(secret_key, b_x5[j])
                ciphertext7[j] = ctx.encrypt(secret_key, b_x6[j])
                ciphertext8[j] = ctx.encrypt(secret_key, b_x7[j])
                ciphertext9[j] = ctx.encrypt(secret_key, b_x8[j])
                ciphertext10[j] = ctx.encrypt(secret_key, b_x9[j])
                ciphertext11[j] = ctx.encrypt(secret_key, b_x10[j])
                ciphertext12[j] = ctx.encrypt(secret_key, b_x11[j])
                ciphertext13[j] = ctx.encrypt(secret_key, b_x12[j])
                ciphertextl[j] = ctx.encrypt(secret_key, b_l[j])
                
                
            #ciphertext1 = ctx.encrypt(secret_key, x)
            #ciphertext2 = ctx.encrypt(secret_key, y) 
            start_time = time.time()
            #ciphertext1.reverse()
            #ciphertext2.reverse()
            #result = addNumbers(ciphertext1, ciphertext2, size)
            #partial_predict=[]

            #print(type(ciphertext1[0][0]))
            
            #wx=[[vm.empty_ciphertext((1,))] for i in range(size)]
            #predict[i]=[[vm.empty_ciphertext((1,))] for i in range(size*2)]
            #temp1=[]
            #temp2=[]
            presult_mul1 = euclidean_distance(ciphertext1,ciphertext1a, size)
            #print("ciphertext one",type(ciphertext1))
            #print("ciphertext one inside",type(ciphertext1[0]))
            ''''temp1=presult_mul1[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)'''
            #print("mul1",twos_comp_val(int(pa,2),len(pa)))
            presult_mul2 = euclidean_distance(ciphertext2,ciphertext2a , size)
            '''temp1=presult_mul2[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)'''
            #print("mul2",twos_comp_val(int(pa,2),len(pa)))
            presult_mul3 = euclidean_distance(ciphertext3,ciphertext3a,  size )
            '''temp1=presult_mul3[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)'''
            #print("mul3",twos_comp_val(int(pa,2),len(pa)))
            presult_mul4 = euclidean_distance(ciphertext4, ciphertext4a,  size)
            '''temp1=presult_mul4[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)'''
            #print("mul4",twos_comp_val(int(pa,2),len(pa)))
            presult_mul4 = euclidean_distance(ciphertext4, ciphertext4a,  size)
            presult_mul5 = euclidean_distance(ciphertext5, ciphertext5a,  size)
            presult_mul6 = euclidean_distance(ciphertext6, ciphertext6a,  size)
            presult_mul7 = euclidean_distance(ciphertext7, ciphertext7a,  size)
            presult_mul8 = euclidean_distance(ciphertext8, ciphertext8a,  size)
            presult_mul9 = euclidean_distance(ciphertext9, ciphertext9a,  size)
            presult_mul10 = euclidean_distance(ciphertext10, ciphertext10a,  size)
            presult_mul11 = euclidean_distance(ciphertext11, ciphertext11a,  size)
            presult_mul12 = euclidean_distance(ciphertext12, ciphertext12a,  size)
            presult_mul13 = euclidean_distance(ciphertext13, ciphertext13a,  size)
            presult_add1 = addNumbers(presult_mul1, presult_mul2,  size)
            '''temp1=presult_add1[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)'''
            #print("firstadd",pa)
            #print("add1",twos_comp_val(int(pa,2),len(pa)))
            presult_add2 = addNumbers(presult_mul3, presult_mul4,  size)
            '''temp1=presult_add2[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)'''
            #print("secondadd",pa)
            #print("add2",twos_comp_val(int(pa,2),len(pa)))
            presult_add2 = addNumbers(presult_mul3, presult_mul4,  size)
            presult_add3 = addNumbers(presult_mul5, presult_mul6,  size)
            presult_add4 = addNumbers(presult_mul7, presult_mul8,  size)
            presult_add5 = addNumbers(presult_mul9, presult_mul10,  size)
            presult_add6 = addNumbers(presult_mul11, presult_mul12,  size)
            #second reduction
            presult_add21 = addNumbers(presult_add1, presult_add2,  size)
            presult_add22 = addNumbers(presult_add3, presult_add4,  size)
            presult_add23 = addNumbers(presult_add5, presult_add6,  size)
            #third reduction

            presult_add31 = addNumbers(presult_add21, presult_add22,  size)
            presult_add32 = addNumbers(presult_add23, presult_mul13,  size)

            dist[i]= addNumbers(presult_add31, presult_add32,  size)
            
            
            '''presult_add1 = addNumbers_approx(presult_mul1, presult_mul2,  size)
            presult_add2 = addNumbers_approx(presult_mul3, presult_mul4,  size)
            presult_add3 = addNumbers_approx(presult_mul5, presult_mul6,  size)
            presult_add4 = addNumbers_approx(presult_mul7, presult_mul8,  size)
            presult_add5 = addNumbers_approx(presult_mul9, presult_mul10,  size)
            presult_add6 = addNumbers_approx(presult_mul11, presult_mul12,  size)
            #second reduction
            presult_add21 = addNumbers_approx(presult_add1, presult_add2,  size)
            presult_add22 = addNumbers_approx(presult_add3, presult_add4,  size)
            presult_add23 = addNumbers_approx(presult_add5, presult_add6,  size)
            #third reduction

            presult_add31 = addNumbers_approx(presult_add21, presult_add22,  size)
            presult_add32 = addNumbers_approx(presult_add23, presult_mul13,  size)

            dist[i][0]= addNumbers_approx(presult_add31, presult_add32,  size)'''
            temp1=dist[i]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("distance is",twos_comp_val(int(pa,2),len(pa)))
           
           
            #print(type(temp))
            #x = twos_complement(deci_x,size)
            label[i]=ciphertextl[:]
            
            '''signbit=vm.gate_copy(wx[len(wx)-1])
            print("signbit is ",ctx.decrypt(secret_key, signbit))
            #print("wx type is",type(wx))
            #print("wx inside is",type(wx[0]))
            bits1 = [ctx.encrypt(secret_key,[False])] #[[vm.empty_ciphertext((1,))] for i in range(size)]
            #bits1=[[vm.empty_ciphertext((1,))] for i in range(1)]
            #print("bits1 datatype",type(bits1))
            wxcpy=[[vm.empty_ciphertext((1,))] for i in range(size)]
            wxcpy=wx[:]
            for i in range(size):
                bits3=vm.gate_copy(wx[i])
                wx[i]=vm.gate_not(bits3)
            wxtwos=make_neg(wx,size)
            x=[[vm.empty_ciphertext((1,))] for i in range(size)]

            for i in range(size):
                x[i]=vm.gate_mux(signbit,wxtwos[i],wxcpy[i])
            m=addNumbers(x,onep,size)
            temp1=m[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("m values is",twos_comp_val(int(pa,2),len(pa)))
            wxten=mulNumbers(wxcpy, eten, secret_key, size, size * 2)
            temp1=wxten[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("wxtenvalue",twos_comp_val(int(pa,2),len(pa)))
            sigmoid=nonRestoringDivision(m,wxten,size)
            temp1=sigmoid[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("divison result",twos_comp_val(int(pa,2),len(pa)))


            sigmoidneg = [[vm.empty_ciphertext((1,))] for i in range(size)]

            for i in range(size):
                bits1=vm.gate_copy(sigmoid[i])
                sigmoidneg[i]=vm.gate_not(bits1)
            sigmoidtwos=addNumbers(sigmoidneg,onep,size)
            temp1=sigmoidtwos[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("sigmoid2s",twos_comp_val(int(pa,2),len(pa))) 
            fsigmoid = [[vm.empty_ciphertext((1,))] for i in range(size)]
            for i in range(size):
                fsigmoid[i]=vm.gate_mux(signbit,sigmoidtwos[i],sigmoid[i])

            temp1=fsigmoid[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            #print("fsigmoid",pa)
            print("fsigmoid",twos_comp_val(int(pa,2),len(pa)))
            #print("fsigmoid",len(fsigmoid))
            temp1=presult_add3[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("thirdadd",pa)
            print("add3",twos_comp_val(int(pa,2),len(pa)))
            #partial_predict = addNumbers(presult_add3, bias,  size)
            #partial_predict.revesrse()'''
            '''result_bits = [ctx.decrypt(secret_key, partial_predict[i]) for i in range(size)]
            result_bits.reverse()
            pa=listToString(result_bits)
            print("partial predict",twos_comp_val(int(pa,2),len(pa)))'''
            '''
            predict1=predict(fsigmoid,secret_key,size)
            predict1.reverse()
            #print(predict1)
            #print(type(predict1))
            #print(type(predict1[0]))
            #result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size*2)]
            result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size)]
            #plain_predict.append(boolListToInt(result_bits))
            pa=listToString(result_bits)
            plain_predict.append(twos_comp_val(int(pa,2),len(pa)))
            #print(" nuFHE multiplication number is : ", plain_predict[i])
            print("i value is",i, "endtime",datetime.now(),"start time",start_time)'''
        print("distance array")
        for m in range(0,train_size):
            temp1=dist[m]
            #temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            x=twos_comp_val(int(pa,2),len(pa))
            print(x)
        for i in range(0, train_size-1):    
            # Last i elements are already in place
            for j in range(0,train_size-i-1):
                #if (arr[j] > arr[j+1]):
                #swap(&arr[j], &arr[j+1]);
                
                temp1=dist[j]
                #temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa=listToString(result_bits)
                x=twos_comp_val(int(pa,2),len(pa))
                temp1=dist[j+1]
                #temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa1=listToString(result_bits)
                y=twos_comp_val(int(pa1,2),len(pa1))
                print("before swap",x,y)
                signbit= ctx.encrypt(secret_key, [False])
                
                signbitnot= ctx.encrypt(secret_key, [False])
                
                dist_min=dist[j][:]
                dist_max=dist[j+1][:]
                signbit=subtractNumbers_2s(dist_min,dist_max, size)
                
                print("sing bit is",ctx.decrypt(secret_key, signbit))
                signbitnot=vm.gate_not(signbit)
                for k in range(size):
                    temp_s1[k]=vm.gate_mux(signbit,dist[j][k],dist[j+1][k])
                    
                    temp_s2[k]=vm.gate_mux(signbit,dist[j+1][k],dist[j][k])
                    
                    temp_s3[k]=vm.gate_mux(signbit,label[j][k],label[j+1][k])
                    
                    temp_s4[k]=vm.gate_mux(signbit,label[j+1][k],label[j][k]) 
                     
                dist[j]=temp_s1[:]
                dist[j+1]=temp_s2[:]
                label[j]=temp_s3[:]
                label[j+1]=temp_s4[:]
                temp1=label[j]
                #temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa=listToString(result_bits)
                print(pa)
                x=twos_comp_val(int(pa,2),len(pa))
                temp1=label[j+1]
                #temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa1=listToString(result_bits)
                print(pa1)
                y=twos_comp_val(int(pa1,2),len(pa1))
                print("after",x,y)
                #print("wx is",pa)
                #print("wx",twos_comp_val(int(pa,2),len(pa)))
        
        print("after soring array is")
        for m in range(train_size):
            temp1=dist[m]
            #temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            x=twos_comp_val(int(pa,2),len(pa))
            print(x)
        print("sorting print over")
        
        
        for i in range(neigh):
            signbitl=label[i][size-1]
            signplain=[]
            signplain.append([ctx.decrypt(secret_key,signbitl)])
            print("signbit is",signplain)
            #signbitc=onep[0]
            #dbit=vm.gate_xnor(signbitl,signbitc)
            
            posc=addNumbers(pos,one,size)
            negc=addNumbers(neg,one,size)
            for j in range(size):
                pos[j]=vm.gate_mux(signbitl,posc[j],pos[j])
                neg[j]=vm.gate_mux(signbitl,neg[j],negc[j])     
        
        temp1=pos[:]
        #temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        result_bits.reverse()
        pa=listToString(result_bits)
        print(pa)
        print("positive count",twos_comp_val(int(pa,2),len(pa)))
        temp1=neg[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        
        pa=listToString(result_bits)
        print(pa)
        print("negative count",twos_comp_val(int(pa,2),len(pa)))
        print("======================")
        pos.reverse()
        neg.reverse()
        signbit=subtractNumbers_2s(pos,neg,size)
        print("count value is decession is",ctx.decrypt(secret_key, signbit))
        result_bits = [ctx.decrypt(secret_key, neg_l[i]) for i in range(size)]
        result_bits.reverse()
        pa=listToString(result_bits)
        print("neg value after decryption=",twos_comp_val(int(pa,2),len(pa)))
        result_bits = [ctx.decrypt(secret_key, pos_l[i]) for i in range(size)]
        result_bits.reverse()
        pa=listToString(result_bits)
        print("positive value after decryption=",twos_comp_val(int(pa,2),len(pa)))
        
        for m in range(size):
          predict1[m]=vm.gate_mux(signbit,neg_l[m],pos_l[m])
        #predict1.reverse()
        #print(predict1)
        #print(type(predict1))
        #print(type(predict1[0]))
        #result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size*2)]
        result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size)]
        #plain_predict.append(boolListToInt(result_bits))
        pa=listToString(result_bits)
        print(pa)
        print("predicted",twos_comp_val(int(pa,2),len(pa)))
        plain_predict.append(twos_comp_val(int(pa,2),len(pa)))
        print("jvalue is ",j,"endtime",datetime.now(),"start time",start_time)   
    #print("start time is",start_time)
    #print("end time",time.time())
    print("endtime",datetime.now(),"start time",start_time)
    #result.reverse()
    for j in range(0,test_size):
        print(" nuFHE multiplication number is actual",lines[j][13],"preicted", plain_predict[j])

560	10	0	1300	2830	10	0	1030	10	16	0	0	30	-1

640	10	0	1280	2630	0	10	1050	10	2	10	10	30	1

460	10	0	1200	2490	0	0	1440	0	8	20	0	30	-1

610	10	30	1340	2340	0	10	1450	0	26	10	20	20	-1

510	0	20	1200	2950	0	0	1570	0	6	20	0	20	1

670	10	0	1200	2370	0	10	710	0	10	10	0	20	-1

430	10	0	1150	3030	0	10	1810	0	12	10	0	20	1

420	0	20	1200	2090	0	10	1730	0	0	10	0	20	1

580	10	0	1000	2340	0	10	1560	0	1	20	10	30	-1

470	10	0	1100	2750	0	0	1180	10	10	10	10	20	-1

670	0	0	1060	2230	0	10	1420	0	3	20	20	20	1

760	0	20	1400	1970	0	20	1160	0	11	10	0	20	1

700	10	10	1560	2450	0	0	1430	0	0	20	0	20	1

390	0	20	1380	2200	0	10	1520	0	0	10	0	20	1

580	0	0	1300	1970	0	10	1310	0	6	10	0	20	1

470	10	20	1300	2530	0	10	1790	0	0	20	0	20	1

440	0	20	1180	2420	0	10	1490	0	3	10	10	20	1

630	10	0	1400	1870	0	0	1440	10	40	20	20	30	-1

630	0	0	1240	1970	0	10	1360	10	0	10	0	20	-1

590	10	0	1640	1760	10	0	900	0	10	10	20	10	-1

570	0	0	1400	2410	0	10	1230	10	2	10	0	30	-1

450	10	30	1100	2640	0	10	1320	0	12	10	0	30	-1

640	10

Exception ignored in: <function Thread.__del__ at 0x7fa6e73b8dd0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/reikna/cluda/cuda.py", line 247, in __del__
  File "/usr/local/lib/python3.7/dist-packages/reikna/cluda/api.py", line 597, in release
  File "/usr/local/lib/python3.7/dist-packages/reikna/cluda/cuda.py", line 242, in _release_specific
pycuda._driver.LogicError: context::pop failed: invalid device context - cannot pop non-current context


signbit is [[array([False])]]
signbit is [[array([ True])]]
0000000000000001
positive count 1
0000000000000010
negative count 2
1000000000000000
CAta in 2s -32768
0100000000000000
catb in 2s 16384
1111111111111111
subtraction for decession -1
count value is decession is [ True]
neg value after decryption= -1
positive value after decryption= 1
1111111111111111
predicted -1
jvalue is  15 endtime 2022-03-09 11:14:28.997756 start time 1646824266.3609326
endtime 2022-03-09 11:14:28.997951 start time 1646824266.3609326
 nuFHE multiplication number is actual 1 preicted -1
time: 11min 34s (started: 2022-03-09 11:02:54 +00:00)


In [ ]:
import time
import random
#import pycuda.autoinit
import nufhe
import numpy as np
import numpy as np
from datetime import datetime
def subtractBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)      #axorb
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry) #axorbxorc
    # And(t1[0], t1[0], t2[0])
    acomp=vm.gate_not(a)   #a'
    abcomp=vm.gate_not(t1)  #axorb'
    t2 = vm.gate_and(acomp, b)
    t3 = vm.gate_and(abcomp, carry)
    #t4=vm.gate_and(t2,t3)
    #t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t2, t3)

    return r
def subtractNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(0,nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    t1=vm.gate_not(ctA[0])
    carry = vm.gate_and(t1, ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
        bitResult = subtractBits(bitResult, ctA[i], ctB[i], carry)
        # Copy(ctRes[i], bitResult[0]);
        ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])

        # Copy(carry[0], bitResult[1])
        carry = nufhe.LweSampleArray.copy(bitResult[1])

    return ctRes

def predict_classification(train, test_row, num_neighbors):
	neighbors = get_neighbors(train, test_row, num_neighbors)
	output_values = [row[-1] for row in neighbors]
	prediction = max(set(output_values), key=output_values.count)
	return prediction
#def bubblesort(distances)

def get_neighbors(train, test_row, num_neighbors):
	distances = list()
	for train_row in train:
		dist = euclidean_distance(test_row, train_row,nbits)
		distances.append((train_row, dist))
	distances.sort(key=lambda tup: tup[1])
	neighbors = list()
	for i in range(num_neighbors):
		neighbors.append(distances[i][0])
	return neighbors




def euclidean_distance(row1, row2,nBits):
    ones= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    distance=  [ctx.encrypt(secret_key, [False]) for _ in range(nBits)]
    c_zero= [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    #distance = 0.0
    '''for i in range(nBits):
        temp=vm.gate_copy(row2[i])
        ones[i]=vm.gate_not(temp)
    neg2=make_neg(ones,nBits)'''
    
    
    result=subtractNumbers(row1,row2,nBits)
    signbit=vm.gate_copy(result[len(result)-1])
    for i in range(nBits):
        temp=vm.gate_copy(result[i])
        ones[i]=vm.gate_not(temp)
    neg2=make_neg(ones,nBits)      
    for i in range(0,nBits):
           distance[i]=vm.gate_mux(signbit,neg2[i],result[i])
    return distance



def make_neg(n,nbits):
    list1=[int(i) for i in range(0,len(n)) ]
    listp=[]
    #one=vm.empty_ciphertext((1,))
    #zero=[vm.empty_ciphertext((1,))]
    #one= [ctx.encrypt(secret_key, [True])]
    #zero= [ctx.encrypt(secret_key, [False])]
    #t2= [ctx.encrypt(secret_key, [False])]
    #one=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    #zero=  [ctx.encrypt(secret_key, [False) for _ in range(nbits)]
    #print("type of one is",type(one))
    #print("typeof zero is",type(zero))
    #print("tupe of one is",one)
    #[vm.empty_ciphertext((1,))]
    '''for i in range(0,len(n)):
        #temp=vm.gate_copy(n[i])
        temp=vm.gate_mux(n[i],zero[0],one[0])
        n[i]=temp[:]'''
            
    
    #print(n)
    one= ctx.encrypt(secret_key, [True])
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(nbits)]
    onep[0]=one 
    testone= [[vm.empty_ciphertext((1,))] for i in range(nbits)]
    testone=onep[:]
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n before",twos_comp_val(int(pa,2),len(pa)))'''
    n=addNumbers(n,testone,nbits)
    '''temp1=n[:]
    temp1.reverse()
    result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(nBits)]
    pa=listToString(result_bits)
    print("n after",twos_comp_val(int(pa,2),len(pa)))'''
    return n


def fixSizeBoolList(decimal,size):
    x = [int(x) for x in bin(decimal)[2:]]
    x = list(map(bool, x))
    x = [False]*(size - len(x)) + x
    pow2 = []
    for i in range(size):
        pow2.append([x[i]])
    pow2.reverse()
    return pow2
def boolListToInt(bitlists):
    out = 0
    for bit in bitlists:
        out = (out << 1) | bit
    return out

def addBits(r, a, b, carry):
    # Xor(t1[0], a, carry[0])
    t1 = vm.gate_xor(a, b)
    # Xor(t2[0], b, carry[0])
    # Xor(r[0], a, t2[0])
    r[0] = vm.gate_xor(t1, carry)
    # And(t1[0], t1[0], t2[0])
    t2 = vm.gate_and(a, carry)
    t3 = vm.gate_and(b, carry)
    t4=vm.gate_and(a,b)
    t5= vm.gate_or(t2,t3)
    # Xor(r[1], carry[0], t1[0])
    r[1] = vm.gate_or(t5, t4)

    return r
def addNumbers(ctA, ctB, nBits):
    ctRes = [[vm.empty_ciphertext((1,))] for i in range(nBits)]
    # carry = vm.empty_ciphertext((1,))
    bitResult = [[vm.empty_ciphertext((1,))] for i in range(2)]
    ctRes[0] = vm.gate_xor(ctA[0], ctB[0])
    # Xor(ctRes[0], ctA[0], ctB[0])
    carry = vm.gate_and(ctA[0], ctB[0])
    # And(carry[0], ctA[0], ctB[0])
    for i in range(1,nBits ):
        if i>0:
            bitResult = addBits(bitResult, ctA[i], ctB[i], carry)
            # Copy(ctRes[i], bitResult[0]);
            ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
            # Copy(carry[0], bitResult[1])
            carry = nufhe.LweSampleArray.copy(bitResult[1])
        else:
            #bitResult = twotoone(bitResult, ctA[i], ctB[i])
            bitResult = threetotwo(bitResult, ctA[i], ctB[i],carry)
            #Copy(ctRes[i], bitResult[0]);
            ctRes[i] = nufhe.LweSampleArray.copy(bitResult[0])
            carry = nufhe.LweSampleArray.copy(bitResult[1])
        
    return ctRes
def Convert_list(string):
    list1=[]
    list1[:0]=string
    print(list1)
    list1=[int(i)for i in list1 ]
    listb=[]
    for i in list1:
        if i==0:
            listb.append([False])
        else:
            listb.append([True])    
    
    #print(listb)
    return listb

def twos_complement(n,nbits):
    a=f"{n & ((1 << nbits) - 1):0{nbits}b}" 
    #print(type(a))
    a=Convert_list(a)
    a.reverse()
    return a
def listToString(s): 
    # initialize an empty string
    list1=[int(i)for i in s ]
    listp=[]
    for i in list1:
        if i==False:
            listp.append('0')
        else:
            listp.append('1')    
    
    #print(listp)
    str1 = "" 
    # traverse in the string  
    s=['delim'.join([str(elem) for elem in sublist]) for sublist in listp]
    #print(s)
    for ele in s: 
        str1 += ele  
    # return string 
    #print(str1)
    return str1
def twos_comp_val(val,bits):
    """compute the 2's complement of int value val"""
    #val=listToString(val)
    

    if (val & (1 << bits - 1)) != 0: # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)        # compute negative value
    return val

def compare_bit(  a, b,  lsb_carry,  tmp): 
    result= ctx.encrypt(secret_key, [False])
    tmp=vm.gate_xnor(a, b)
    result=vm.gate_mux(tmp,lsb_carry, a)
    return result


def minimum(  a,  b,  nb_bits): 
    tmps1= ctx.encrypt(secret_key, [False])
    tmps2= ctx.encrypt(secret_key, [True])
    #initialize the carry to 0
    #run the elementary comparator gate n times
    for i in range(nb_bits):
        tmps1= compare_bit(a[i],b[i],tmps1,tmps2)
    #tmps[0] is the result of the comparaison: 0 if a is larger, 1 if b is larger
    #select the max and copy it to the result
    return tmps1

    
        
def predict(ctA,secret_key, output_bits):
    zero = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    onen=  [ctx.encrypt(secret_key, [True]) for _ in range(output_bits)]        
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    one= ctx.encrypt(secret_key, [True])
    temp= ctx.encrypt(secret_key, [True])
    temp=ctA[output_bits-1]
    comp_res= ctx.encrypt(secret_key, [True])
    onep[0]=one
    ctRes = [ctx.encrypt(secret_key, [False]) for _ in range(output_bits)]
    # Copy(ctRes[i], bitResult[0]);
    #comp_res= minimum(ctA,zero,output_bits)
    #temp=
    #comp_res=
    
    
    
    for i in range(output_bits):
        ctRes[i] = vm.gate_mux(temp,onen[i],onep[i])  
    # Copy(carry[0], bitResult[1])
    return ctRes


def dist_calc(line,line1,nbits):
    #print(line)
    print(type(list))
    
    return wx,b_l














if __name__ == '__main__':    
    ctx = nufhe.Context()
    secret_key, cloud_key = ctx.make_key_pair()
    vm = ctx.make_virtual_machine(cloud_key)
    size=16
    onep=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    onen=  [ctx.encrypt(secret_key, [True]) for _ in range(size)]
    temp_dist=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    temp_label=  [ctx.encrypt(secret_key, [False]) for _ in range(size)]
    one= ctx.encrypt(secret_key, [True])
    zero=ctx.encrypt(secret_key, [False])
    for k in range(size):
      onen[i]=one
    onep[0]=one
    onep[0]=vm.gate_copy(one)
    with open('/content/drive/MyDrive/train.txt') as f:
        lines = []
        for line in f:
            lines.append([int(v) for v in line.split()])
    with open('/content/drive/MyDrive/test.txt') as f:
        lines1 = []
        for line in f:
            lines1.append([int(v) for v in line.split()])
    bits = [[False] for i in range(size - 2)]
    zeros = [[False] for i in range(size)]
    train_size= int(input("Please enter train a string:\n"))
    test_size=int(input("Please enter test data a string:\n"))
    neigh=int(input("Please enter no of neighbours:\n"))
    

    '''b_x0=[]
    b_x1=[]
    b_x2=[]
    b_x3=[]
    for i in range(test_size):
        temp=int(lines[i][0])
        #print(type(temp))
        b_x0.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][1])
        b_x1.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][2])
        b_x2.append(fixSizeBoolList(temp,size))
        temp=int(lines[i][3])
        b_x3.append(fixSizeBoolList(temp,size))'''
    #print(b_x0,b_x1,b_x2,b_x3)
    #-3.5133618, -4.2616279 , 10.59763759,  6.17913037, -0.1758181
    #w_p=[-15,-18 ,-78 ,138 , 62] # scratch x/1+|x|
    w_p=[-1,-3,4,2,0] # sk learn x/1+|x|
    #w_p=[4,-8 ,22 ,9 , -1] sigmoid
    #w_p=[-3, -4 , 10,  6, -1] svm
    
    #print(type(x[0]))
    #print(x)
    #b_y = fixSizeBoolList(deci_y,size)
    #print(type(y[0]))
    #print(y)
    #x.reverse()
    #print(x)
    #y.reverse()
    #print(y)
    featuresize=5
    '''w=[]
    w_b=[]
    for i in range(featuresize-1):
        w.append([[vm.empty_ciphertext((1,))] for i in range(size)])
        w_b.append(twos_complement(w_p[i],size))
        for j in range(size):
            w[i][j] = ctx.encrypt(secret_key, w_b[i][j])
    bias=[[vm.empty_ciphertext((1,))] for i in range(size)]
    bias_b=twos_complement(w_p[4],size)'''
    '''eten=[[vm.empty_ciphertext((1,))] for i in range(size)]
    eten_b=twos_complement(10,size)
    for i in range(size):
        bias[i]=ctx.encrypt(secret_key, bias_b[i])
        eten[i]=ctx.encrypt(secret_key, eten_b[i])
    '''
    #train_data = pd.read_csv('/home/pradeep/Desktop/train.txt',sep=" ",header = None)
    #test_data=pd.read_csv('/home/pradeep/Desktop/test.txt',sep=" ",header = None)
    dist=[]
    for i in range(train_size):
        dist.append([[ctx.encrypt(secret_key, [False]) for i in range(16)] for j in range(2)]) 
    
    plain_predict=[]
    start_time = datetime.now()
    #dist=[[vm.empty_ciphertext((1,))] for i in range(train_size) for j in range(size)]
    for j in range(test_size):
        print("j values is",j)
        print("################################")
        for i in range(train_size):
            print("i value is ",i)
            print(lines[i][0],lines[i][1],lines[i][2],lines[i][3])
            print(lines1[j][0],lines1[j][1],lines1[j][2],lines1[j][3])
            temp=int(lines[i][0])
            b_x0=twos_complement(temp,size)
            temp=int(lines[i][1])
            b_x1=twos_complement(temp,size)
            temp=int(lines[i][2])
            b_x2=twos_complement(temp,size)
            temp=int(lines[i][3])
            b_x3=twos_complement(temp,size)
            temp=int(lines[i][4])
            b_l=twos_complement(temp,size)
            temp1=int(lines1[j][0])
            b_y0=twos_complement(temp1,size)
            temp1=int(lines1[j][1])
            b_y1=twos_complement(temp1,size)
            temp1=int(lines1[j][2])
            b_y2=twos_complement(temp1,size)
            temp1=int(lines1[j][3])
            b_y3=twos_complement(temp1,size)
            ciphertext1=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext2=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext3=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext4=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertextl=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext1a=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext2a=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext3a=[[vm.empty_ciphertext((1,))] for i in range(size)]
            ciphertext4a=[[vm.empty_ciphertext((1,))] for i in range(size)]
            for j in range(size):
                ciphertext1[j] = ctx.encrypt(secret_key, b_x0[j])
                ciphertext2[j] = ctx.encrypt(secret_key, b_x1[j])
                ciphertext3[j] = ctx.encrypt(secret_key, b_x2[j])
                ciphertext4[j] = ctx.encrypt(secret_key, b_x3[j])
                ciphertextl[j] = ctx.encrypt(secret_key, b_l[j])
                ciphertext1a[j] = ctx.encrypt(secret_key, b_y0[j])
                ciphertext2a[j] = ctx.encrypt(secret_key, b_y1[j])
                ciphertext3a[j] = ctx.encrypt(secret_key, b_y2[j])
                ciphertext4a[j] = ctx.encrypt(secret_key, b_y3[j])
            #ciphertext1 = ctx.encrypt(secret_key, x)
            #ciphertext2 = ctx.encrypt(secret_key, y) 
            start_time = time.time()
            #ciphertext1.reverse()
            #ciphertext2.reverse()
            #result = addNumbers(ciphertext1, ciphertext2, size)
            #partial_predict=[]

            #print(type(ciphertext1[0][0]))
            temp1 = [[vm.empty_ciphertext((1,))] for i in range(size)]
            temp2 = [[vm.empty_ciphertext((1,))] for i in range(size)]
            #wx=[[vm.empty_ciphertext((1,))] for i in range(size)]
            #predict[i]=[[vm.empty_ciphertext((1,))] for i in range(size*2)]
            #temp1=[]
            #temp2=[]
            presult_mul1 = euclidean_distance(ciphertext1,ciphertext1a, size)
            #print("ciphertext one",type(ciphertext1))
            #print("ciphertext one inside",type(ciphertext1[0]))
            temp1=presult_mul1[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("mul1",twos_comp_val(int(pa,2),len(pa)))
            presult_mul2 = euclidean_distance(ciphertext2,ciphertext2a , size)
            temp1=presult_mul2[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("mul2",twos_comp_val(int(pa,2),len(pa)))
            presult_mul3 = euclidean_distance(ciphertext3,ciphertext3a,  size )
            temp1=presult_mul3[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("mul3",twos_comp_val(int(pa,2),len(pa)))
            presult_mul4 = euclidean_distance(ciphertext4, ciphertext4a,  size)
            temp1=presult_mul4[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("mul4",twos_comp_val(int(pa,2),len(pa)))
            presult_add1 = addNumbers(presult_mul1, presult_mul2,  size)
            temp1=presult_add1[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("firstadd",pa)
            print("add1",twos_comp_val(int(pa,2),len(pa)))
            presult_add2 = addNumbers(presult_mul3, presult_mul4,  size)
            temp1=presult_add2[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            #print("secondadd",pa)
            print("add2",twos_comp_val(int(pa,2),len(pa)))
            dist[i][0]= addNumbers(presult_add1, presult_add2,  size)
            #print(type(temp))
            #x = twos_complement(deci_x,size)
            dist[i][1]=ciphertextl[:]
            
            '''signbit=vm.gate_copy(wx[len(wx)-1])
            print("signbit is ",ctx.decrypt(secret_key, signbit))
            #print("wx type is",type(wx))
            #print("wx inside is",type(wx[0]))
            bits1 = [ctx.encrypt(secret_key,[False])] #[[vm.empty_ciphertext((1,))] for i in range(size)]
            #bits1=[[vm.empty_ciphertext((1,))] for i in range(1)]
            #print("bits1 datatype",type(bits1))
            wxcpy=[[vm.empty_ciphertext((1,))] for i in range(size)]
            wxcpy=wx[:]
            for i in range(size):
                bits3=vm.gate_copy(wx[i])
                wx[i]=vm.gate_not(bits3)
            wxtwos=make_neg(wx,size)
            x=[[vm.empty_ciphertext((1,))] for i in range(size)]

            for i in range(size):
                x[i]=vm.gate_mux(signbit,wxtwos[i],wxcpy[i])
            m=addNumbers(x,onep,size)
            temp1=m[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("m values is",twos_comp_val(int(pa,2),len(pa)))
            wxten=mulNumbers(wxcpy, eten, secret_key, size, size * 2)
            temp1=wxten[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("wxtenvalue",twos_comp_val(int(pa,2),len(pa)))
            sigmoid=nonRestoringDivision(m,wxten,size)
            temp1=sigmoid[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("divison result",twos_comp_val(int(pa,2),len(pa)))


            sigmoidneg = [[vm.empty_ciphertext((1,))] for i in range(size)]

            for i in range(size):
                bits1=vm.gate_copy(sigmoid[i])
                sigmoidneg[i]=vm.gate_not(bits1)
            sigmoidtwos=addNumbers(sigmoidneg,onep,size)
            temp1=sigmoidtwos[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("sigmoid2s",twos_comp_val(int(pa,2),len(pa))) 
            fsigmoid = [[vm.empty_ciphertext((1,))] for i in range(size)]
            for i in range(size):
                fsigmoid[i]=vm.gate_mux(signbit,sigmoidtwos[i],sigmoid[i])

            temp1=fsigmoid[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            #print("fsigmoid",pa)
            print("fsigmoid",twos_comp_val(int(pa,2),len(pa)))
            #print("fsigmoid",len(fsigmoid))
            temp1=presult_add3[:]
            temp1.reverse()
            result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
            pa=listToString(result_bits)
            print("thirdadd",pa)
            print("add3",twos_comp_val(int(pa,2),len(pa)))
            #partial_predict = addNumbers(presult_add3, bias,  size)
            #partial_predict.revesrse()'''
            '''result_bits = [ctx.decrypt(secret_key, partial_predict[i]) for i in range(size)]
            result_bits.reverse()
            pa=listToString(result_bits)
            print("partial predict",twos_comp_val(int(pa,2),len(pa)))'''
            '''
            predict1=predict(fsigmoid,secret_key,size)
            predict1.reverse()
            #print(predict1)
            #print(type(predict1))
            #print(type(predict1[0]))
            #result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size*2)]
            result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size)]
            #plain_predict.append(boolListToInt(result_bits))
            pa=listToString(result_bits)
            plain_predict.append(twos_comp_val(int(pa,2),len(pa)))
            #print(" nuFHE multiplication number is : ", plain_predict[i])
            print("i value is",i, "endtime",datetime.now(),"start time",start_time)'''
        for i in range( train_size-1):    
            # Last i elements are already in place
            for j in range(train_size-i-1):
                #if (arr[j] > arr[j+1]):
                #swap(&arr[j], &arr[j+1]);
                temp=subtractNumbers(dist[j][0],dist[j+1][0], size)
                signbit= ctx.encrypt(secret_key, [False])
                signbitnot= ctx.encrypt(secret_key, [False])
                signbit=temp[size-1]
                signbitnot=vm.gate_not(signbit)
                for k in range(size):
                    dist[j][0][k]=vm.gate_mux(signbit,dist[j][0][k],dist[j+1][0][k])
                    dist[j+1][0][k]=vm.gate_mux(signbit,dist[j+1][0][k],dist[j][0][k])
                    dist[j][1][k]=vm.gate_mux(signbit,dist[j][1][k],dist[j+1][1][k])
                    dist[j+1][1][k]=vm.gate_mux(signbit,dist[j+1][1][k],dist[j][1][k]) 
                temp1=dist[j][0]
                temp1.reverse()
                result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
                pa=listToString(result_bits)
                print("wx is",pa)
                print("wx",twos_comp_val(int(pa,2),len(pa)))
        pos=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
        neg=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
        predict1=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
        posc=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
        negc=[ctx.encrypt(secret_key, [False]) for _ in range(size)]
        for i in range(neigh):
            signbitl=dist[i][1][size-1]
            signplain=[]
            signplain.append([ctx.decrypt(secret_key,signbitl)])
            print("signbit is",signplain)
            signbitc=onep[size-1]
            dbit=vm.gate_xnor(signbitl,signbitc)
            posc=pos[:]
            negc=neg[:]
            pos=make_neg(pos,size)
            neg=make_neg(neg,size)
            for j in range(size):
                pos[j]=vm.gate_mux(dbit,posc[j],pos[j])
                neg[j]=vm.gate_mux(dbit,neg[j],negc[j])     
        temp=subtractNumbers(pos,neg,size)
        temp1=pos[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("positive count",twos_comp_val(int(pa,2),len(pa)))
        temp1=neg[:]
        temp1.reverse()
        result_bits = [ctx.decrypt(secret_key, temp1[i]) for i in range(size)]
        pa=listToString(result_bits)
        print("negative count",twos_comp_val(int(pa,2),len(pa)))
        
        
        signbit=temp[size-1]
        predict1[i]=vm.gate_mux(signbit,onen[i],onep[i])
        predict1.reverse()
        #print(predict1)
        #print(type(predict1))
        #print(type(predict1[0]))
        #result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size*2)]
        result_bits = [ctx.decrypt(secret_key, predict1[i]) for i in range(size)]
        #plain_predict.append(boolListToInt(result_bits))
        pa=listToString(result_bits)
        plain_predict.append(twos_comp_val(int(pa,2),len(pa)))
        print("jvalue is ",j,"endtime",datetime.now(),"start time",start_time)   
    #print("start time is",start_time)
    #print("end time",time.time())
    print("endtime",datetime.now(),"start time",start_time)
    #result.reverse()
    for j in range(test_size):
        print(" nuFHE multiplication number is actual",lines[j][4],"preicted", plain_predict[j])

ModuleNotFoundError: ignored

In [ ]:
import pandas as pd


0    51
1    25
2    30
3    11
4     1
Name: 0, dtype: int64
0    47
1    32
2    13
3     2
4    -1
Name: 1, dtype: int64
0    57
1    28
2    41
3    13
4     1
Name: 2, dtype: int64
0    46
1    31
2    15
3     2
4    -1
Name: 3, dtype: int64
0    50
1    36
2    14
3     2
4    -1
Name: 4, dtype: int64
0    56
1    27
2    42
3    13
4     1
Name: 5, dtype: int64
0    57
1    30
2    42
3    12
4     1
Name: 6, dtype: int64
0    50
1    34
2    15
3     2
4    -1
Name: 7, dtype: int64
0    44
1    29
2    14
3     2
4    -1
Name: 8, dtype: int64
0    56
1    30
2    41
3    13
4     1
Name: 9, dtype: int64


In [ ]:
import time
import random
#import pycuda.autoinit
import nufhe
import numpy as np
import numpy as np
from datetime import datetime
ctx = nufhe.Context()
secret_key, cloud_key = ctx.make_key_pair()
vm = ctx.make_virtual_machine(cloud_key)
size=16
dist=[]
for i in range(2):
        dist.append([[ctx.encrypt(secret_key, [False]) for i in range(16)] for j in range(2)])
print(dist[0][1])        

[<nufhe.lwe.LweSampleArray object at 0x7f6af8502df0>, <nufhe.lwe.LweSampleArray object at 0x7f6b103c95b0>, <nufhe.lwe.LweSampleArray object at 0x7f6af82192b0>, <nufhe.lwe.LweSampleArray object at 0x7f6af8219c40>, <nufhe.lwe.LweSampleArray object at 0x7f6af8247eb0>, <nufhe.lwe.LweSampleArray object at 0x7f6af8342310>, <nufhe.lwe.LweSampleArray object at 0x7f6af84e7e50>, <nufhe.lwe.LweSampleArray object at 0x7f6af842ca60>, <nufhe.lwe.LweSampleArray object at 0x7f6af842c460>, <nufhe.lwe.LweSampleArray object at 0x7f6af84a58b0>, <nufhe.lwe.LweSampleArray object at 0x7f6af84a5f40>, <nufhe.lwe.LweSampleArray object at 0x7f6af84a5bb0>, <nufhe.lwe.LweSampleArray object at 0x7f6af8bfa2b0>, <nufhe.lwe.LweSampleArray object at 0x7f6af8439790>, <nufhe.lwe.LweSampleArray object at 0x7f6af811d730>, <nufhe.lwe.LweSampleArray object at 0x7f6af832bf40>]
